Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: it's a harder task, and the data is a lot less 'clean' than MNIST.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [2]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print 'Found and verified', filename
  else:
    raise Exception(
      'Failed to verify' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [4]:
num_classes = 10

def extract(filename):
  tar = tarfile.open(filename)
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  print('Extracting data for %s. This may take a while. Please wait.' % root)
  sys.stdout.flush()
  tar.extractall()
  tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root)) if d != '.DS_Store']
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = extract(train_filename)
test_folders = extract(test_filename)

NameError: name 'train_filename' is not defined

---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

---

In [7]:
import random
from IPython.display import display, Image

print os.getcwd()
train_folders = os.path.join(os.getcwd(),'notMNIST_large')
test_folders = os.path.join(os.getcwd(),'notMNIST_small')

train_folders = os.listdir(train_folders)
test_folders = os.listdir(test_folders)
train_folders = [os.path.join(os.getcwd(),'notMNIST_large',i) for i in train_folders]
test_folders = [os.path.join(os.getcwd(),'notMNIST_small',i) for i in test_folders]

#display random file from each DIR - 'A' through 'J'
for i in train_folders[1:]:
    fullDirPath = os.path.join(train_folders,i)
    listFiles = os.listdir(fullDirPath)
    randFileIdx = random.randint(1,len(listFiles))
    print 'File number %d: '%(randFileIdx)
    randFilePath = os.path.join(fullDirPath, listFiles[randFileIdx])
    display(Image(filename=randFilePath))
    


/Users/voladoddi/Downloads
File number 41545: 


File number 16298: 


File number 38639: 


File number 52537: 


File number 9867: 


File number 34175: 


File number 50472: 


File number 45334: 


File number 350: 


File number 7316: 


Now let's load the data in a more manageable format.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road. The labels will be stored into a separate array of integers 0 through 9.

A few images might not be readable, we'll just skip them.

In [9]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load(data_folders, min_num_images, max_num_images):
  data_folders = data_folders[1:]
  print data_folders
  dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size), dtype=np.float32)
  labels = np.ndarray(shape=(max_num_images), dtype=np.int32)
  label_index = 0
  image_index = 0
  for folder in data_folders:
    print(folder)
    for image in os.listdir(folder):
      if image_index >= max_num_images:
        raise Exception('More images than expected: %d >= %d' % (
          image_index, max_num_images))
      image_file = os.path.join(folder, image)
      try:
        image_data = (ndimage.imread(image_file).astype(float) -
                      pixel_depth / 2) / pixel_depth
        if image_data.shape != (image_size, image_size):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))
        dataset[image_index, :, :] = image_data
        labels[image_index] = label_index
        image_index += 1
      except IOError as e:
        print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    label_index += 1
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  labels = labels[0:num_images]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' % (
        num_images, min_num_images))
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  print('Labels:', labels.shape)
  return dataset, labels


train_dataset, train_labels = load(train_folders, 450000, 550000)
test_dataset, test_labels = load(test_folders, 18000, 20000)

['/Users/voladoddi/Downloads/notMNIST_large/A', '/Users/voladoddi/Downloads/notMNIST_large/B', '/Users/voladoddi/Downloads/notMNIST_large/C', '/Users/voladoddi/Downloads/notMNIST_large/D', '/Users/voladoddi/Downloads/notMNIST_large/E', '/Users/voladoddi/Downloads/notMNIST_large/F', '/Users/voladoddi/Downloads/notMNIST_large/G', '/Users/voladoddi/Downloads/notMNIST_large/H', '/Users/voladoddi/Downloads/notMNIST_large/I', '/Users/voladoddi/Downloads/notMNIST_large/J']
/Users/voladoddi/Downloads/notMNIST_large/A
('Could not read:', '/Users/voladoddi/Downloads/notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png', ':', IOError("cannot identify image file '/Users/voladoddi/Downloads/notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png'",), "- it's ok, skipping.")
('Could not read:', '/Users/voladoddi/Downloads/notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png', ':', IOError("cannot identify image file '/Users/voladoddi/Downloads/notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3R

---
Problem 2
---------

Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

---

In [11]:
#for j in range(0, 9):
#    f = [i for i in train_labels if i==j]
#    print len(f)

#PROBLEM: Use pyplot to display images from training data -- random, one image. Use FOR-LOOP for multiple images.
plt.imshow(train_dataset[np.random.randint(train_dataset.shape[0])],cmap='Greys')
plt.show()

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.

In [118]:
####Understand the way permutation works below.
'''Prints length of labels vec. '''
#print train_labels.shape[0] 
'''Prints a PERMUTED version of the array passed into the function.'''
#print np.random.permutation([0,1,2]) 
'''Print some labels to check if randomized'''
#print train_labels[0:10]
#print test_labels[0:10]

'Print some labels to check if randomized'

In [12]:
np.random.seed(133)
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

---
Problem 4
---------
Another check: we expect the data to be balanced across classes. Verify that.

---

In [13]:
'''In other words, do we have more or less
1) equal #training samples across all training classes ? 
2) equal #testing samples across all testing classes? '''

'''Approach 1: for each label#, check how many samples (or associated labels exist)'''
#for j in range(train_labels.min(),train_labels.max()):
#    f = [i for i in train_labels if i==j]
#    print len(f)

'''Approach 2: Visualize - use pyplot'''
plt.subplots(1,2)
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.hist(train_labels)
ax1.set_title('train labels dist')
ax2.hist(test_labels)
ax2.set_title('test labels dist')

#plt.hist(train_labels)
plt.show()

Prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune train_size as needed.

Also create a validation dataset for hyperparameter tuning.

In [14]:
train_size = 200000
valid_size = 10000

valid_dataset = train_dataset[:valid_size,:,:]
valid_labels = train_labels[:valid_size]
train_dataset = train_dataset[valid_size:valid_size+train_size,:,:]
train_labels = train_labels[valid_size:valid_size+train_size]
print('Training', train_dataset.shape, train_labels.shape)
print('Validation', valid_dataset.shape, valid_labels.shape)

('Training', (200000, 28, 28), (200000,))
('Validation', (10000, 28, 28), (10000,))


Finally, let's save the data for later reuse:

In [15]:
pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': train_dataset,
    'train_labels': train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [16]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

('Compressed pickle size:', 718193801)


/Users/voladoddi/Downloads


---
Problem 5
---------

By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it.
Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.
---

In [17]:
pickle_file = 'notMNIST.pickle'
try:
    f = open(pickle_file,'rb')
    contents = pickle.load(f)
    f.close()
except Exception as e:
    print e 

train_dataset = contents['train_dataset']
train_labels = contents['train_labels']
valid_dataset = contents['valid_dataset']
valid_labels = contents['valid_labels']
test_dataset  = contents['test_dataset']
test_labels = contents['test_labels']

count_test = 0
count_valid = 0

#for i in train_dataset:
    
    #if i in valid_dataset:
    #    count_valid = count_valid + 1
    #elif i in test_dataset:
    #    count_test = count_test + 1
    #print count_valid, count_test



In [166]:
''' Means could be duplicated without data being duplicated, 
this is a long and a bore of an approach. But it was fun nevertheless.'''

'''
from operator import itemgetter
def getMean(x):
    return np.mean(x)
def setSampleLabel(sampleArr,labelArr):
    orderedList = []
    for i,j in zip(sampleArr,labelArr):
        orderedList.append((i,j))
    return orderedList

def find_dups(tuple1, tuple2):
    count = [i for i in tuple1 if i in tuple2]
    return len(count)
    
            
#Below steps can be put into one function, just putting down stuff for now
#Tuple train set and labels
trainMean = [getMean(i) for i in train_dataset]
tTrain = setSampleLabel(trainMean,train_labels)
sortedTrain = sorted(tTrain, key=itemgetter(1))

#Tuple valid set and labels
validMean = [getMean(i) for i in valid_dataset]
tValid = setSampleLabel(validMean,valid_labels)
sortedValid = sorted(tValid, key=itemgetter(1))
                     
#Tuple test set and labels
testMean = [getMean(i) for i in test_dataset]
tTest = setSampleLabel(testMean,test_labels)
sortedTest = sorted(tTest, key=itemgetter(1))


dupsTrainingValid = find_dups(sortedValid,sortedTrain)
print '%d pictures in Validation set (%d) are in Train set (%d) as well. ' %(dupsTrainingValid, len(valid_dataset),len(train_dataset))

dupsTestValid = find_dups(sortedValid,sortedTest)
print '%d pictures in Validation set are in Test set(%d) as well.'   %(dupsTestValid, len(test_dataset))

dupsTrainTest = find_dups(sortedTest, sortedTrain)
print '%d pictures in Train set are in Train set as well.' %(dupsTrainTest)
'''


1900 pictures in Validation set (10000) are in Train set (200000) as well. 
346 pictures in Validation set are in Test set(18724) as well.
3898 pictures in Train set are in Train set as well.


---
Problem 6
---------

Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!

---

In [192]:


model = LogisticRegression(multi_class="multinomial", solver="lbfgs")
#trainMean = np.asarray(trainMean)
#print trainMean.shape
#testMean = [getMean(i) for i in test_dataset]
#testMean = np.asarray(testMean)
#print testMean.shape

model.fit(trainMean.reshape(-1,1), train_labels).score(testMean.reshape(-1,1),test_labels)

(200000,)
(18724,)


0.13976714377269814

In [25]:
def trainForN(ntrain):
    traindata = train_dataset[:ntrain].reshape(-1, train_dataset.shape[1]*train_dataset.shape[2])
    trainlabels = train_labels[:ntrain]
    
    testdata = test_dataset.reshape(-1,test_dataset.shape[1]*test_dataset.shape[2])
    
    return traindata, trainlabels, testdata

model = LogisticRegression(multi_class="multinomial", solver = 'lbfgs')
traindata, trainlabels, testdata = trainForN(1000)
model.fit(traindata, trainlabels).score(testdata, test_labels)


0.8409527878658406